## 1.从衰变数据中生成mother-daughter的 $\gamma-\gamma$关联信息

生成test.h 和 test.C

root -l decay46_123_all.root 

[root 0] tree->MakeClass("test"); 

In [1]:
.L test.C 

In [2]:
class ana : public test // 从test类中继承其成员变量和成员函数
{
 public:
   Double_t dt0,dt1;
   Int_t ahit0,ahit1;
   Int_t chit0,chit1;
   Int_t chit01;
   Long64_t bts0;
   Double_t ae0[1000],ae1[1000];
   Double_t cxe0[1000],cye0[1000];
   double_t cxe1[1000],cye1[1000];
   Double_t cde0[1000],cde1[1000];
   ana(TTree* tree): test(tree) {} 
   virtual ~ana() {};
    
   virtual void BranchDelayed(TTree *tree){
   if(!tree)return;
    //decay from mother
   tree->Branch("dt0", &dt0,"dt0/D");
   tree->Branch("ahit0", &ahit0,"ahit0/I");
   tree->Branch("ae0", &ae0,"ae0[ahit0]/D");//signle gamma
   tree->Branch("chit0", &chit0,"chit0/I");
   tree->Branch("cxe0", &cxe0,"cxe0[chit0]/D");//gamma-gamma
   tree->Branch("cye0", &cye0,"cye0[chit0]/D");

    //decay from daughter
   tree->Branch("dt1", &dt1,"dt1/D");
   tree->Branch("ahit1", &ahit1,"ahit1/I");//single gamma
   tree->Branch("ae1", &ae1,"ae0[ahit1]/D");
   tree->Branch("chit1", &chit1,"chit1/I");
   tree->Branch("cxe1", &cxe1,"cxe1[chit1]/D");//gamma-gamma
   tree->Branch("cye1", &cye1,"cye1[chit1]/D");
     
    //mother gamma -daughter gamma 
   tree->Branch("chit01", &chit01,"chit01/I");
   tree->Branch("cde0", &cde0,"cde0[chit01]/D");
   tree->Branch("cde1", &cde1,"cde1[chit01]/D");
 };
 //load selected branches - much faster than tree->GetEntry();   
 virtual void GetEvents(Long64_t entry){
      b_decaytime->GetEntry(entry);
      b_bts->GetEntry(entry);
      b_gts->GetEntry(entry);
      b_ahit->GetEntry(entry);
      b_ae->GetEntry(entry);
      b_at->GetEntry(entry);
      b_cahit->GetEntry(entry);
      b_caxe->GetEntry(entry);
      b_caye->GetEntry(entry);
      b_caxt->GetEntry(entry);
      b_cayt->GetEntry(entry); 
  };
    
//total gamma
 virtual void GetAe(int &ahit1, double *ae1)
 {
   int n=0;
   for(int i=0;i<ahit;i++) {
     if(at[i]>0 && at[i]<1000 ) {
       ae1[n]=ae[i];
       n++;
     }
   }
   ahit1=n;
 };
    
// gamma-gamma coincidence
 virtual void GetCge(int &chit, double *cxe, double * cye)
 {
   int j=0;
   for(int i=0;i<cahit;i++) {
     if(abs(caxt[i]-cayt[i])<200 && caxt[i]>0 &&caxt[i]<1000 && cayt[i]>0&& cayt[i]<1000) {
       cxe[j]=caxe[i];
       cye[j]=caye[i];
       j++;
     }
   }
  chit=j;    
 };
    
//mother-daughter correlation
virtual void GetAe0Ae1(int &chit01, double *cae0, double *cae1 )
 {
   int n=0;
   for(int i=0;i<ahit0;i++) {
     for(int j=0;j<ahit1;j++) {
       cae0[n]=ae0[i];
       cae1[n]=ae1[j];
       n++;
     }
   }
   chit01=n;
 };
    
//analysis
virtual void Analysis(TTree *tree)
{
    BranchDelayed(tree);
    Long64_t nentries = fChain->GetEntriesFast();
    for (Long64_t jentry0=0; jentry0<nentries;jentry0++){
      GetEvents(jentry0);
      if(decaytime<1000 ) {//deaytime<3*T1/2, decay from mother.
          //decay from mother
            bts0=bts;  //get timestamp of beam
            dt0=decaytime;
            GetAe(ahit0,ae0);
            if(ahit0==0) continue;
            GetCge(chit0,cxe0,cye0);
            //decay from daughter.
          for (Long64_t jentry1=jentry0+1; jentry1<nentries;jentry1++){
                GetEvents(jentry1);
                if(bts > bts0) break;//same pixel
                if(bts==bts0 && decaytime-dt0<4000) { //deaytime-dt0<3*T1/2, decay from daughter.
                      dt1=decaytime;
                      GetAe(ahit1,ae1);
                      GetCge(chit1,cxe1,cye1);
                      GetAe0Ae1(chit01,cde0,cde1);//mother -daughter
                      if(ahit1!=0) tree->Fill();
                }// if
          }//jentry1 loop
     }//if
     if(jentry0%50000==0)cout<<".";
    }//jentry0 loop
    cout<<endl;
    cout<<"Done!"<<endl;
  };   
};


In [3]:
TFile *fin=new TFile("../gammaData/decay46_123_all.root");
TTree *treein=(TTree*) fin->Get("tree");
ana *ana1 = new ana(treein);

In [4]:
TFile *fout=new TFile("d46_123.root","RECREATE");
TTree *treeout=new TTree("tree","delayed gamma-gamma");

In [ ]:
ana1->Analysis(treeout);

.......................................................................................................

In [ ]:
fout->Write();
fin->Close();
fout->Close();